In [1]:
import requests
import pandas as pd

API_KEY = '0a61f1e1288fa87a270ad82b0d37bed3'
SPORT = 'upcoming'

def fetch_odds_data():
    url = f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds'
    params = {
        'apiKey': API_KEY,
        'regions': 'us',
        'markets': 'h2h',
        'oddsFormat': 'decimal',
        'dateFormat': 'iso'
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Failed to fetch data: {response.status_code}')

def calculate_implied_prob(odds):
    return 1 / odds

def calculate_ev(odds, prob):
    return (odds * prob) - (1 - prob)

def extract_odds_data(odds_data):
    data = []
    for event in odds_data:
        for bookmaker in event['bookmakers']:
            for market in bookmaker['markets']:
                for outcome in market['outcomes']:
                    data.append({
                        'sport': event['sport_title'],
                        'teams': event['home_team'] + ' vs ' + event['away_team'],
                        'bookmaker': bookmaker['title'],
                        'market': market['key'],
                        'outcome': outcome['name'],
                        'odds': outcome['price'],
                    })
    return pd.DataFrame(data)

# Step 1: Fetch data from The Odds API
odds_data = fetch_odds_data()

# Step 2: Extract relevant data
odds_df = extract_odds_data(odds_data)

# Step 3: Calculate implied probabilities
odds_df['implied_prob'] = calculate_implied_prob(odds_df['odds'])

# Step 4: Calculate EV
odds_df['ev'] = calculate_ev(odds_df['odds'], odds_df['implied_prob'])

# Step 5: Filter positive EV bets
positive_ev_bets = odds_df[odds_df['ev'] > 0]

# Convert implied probability to percentage
positive_ev_bets['implied_prob_percentage'] = positive_ev_bets['implied_prob'] * 100

# Organize and format the table
positive_ev_bets = positive_ev_bets[['sport', 'teams', 'market', 'outcome', 'bookmaker', 'odds', 'implied_prob_percentage', 'ev']]
positive_ev_bets.columns = ['Sport', 'Teams', 'Market', 'Outcome', 'Bookmaker', 'Odds', 'Win Probability (%)', 'Expected Value']

# Display the table using pandas' built-in capabilities
print(positive_ev_bets.to_string(index=False))


         Sport                                     Teams Market               Outcome    Bookmaker  Odds  Win Probability (%)  Expected Value
          WNBA         New York Liberty vs Atlanta Dream    h2h         Atlanta Dream   DraftKings 13.00             7.692308        0.076923
          WNBA         New York Liberty vs Atlanta Dream    h2h      New York Liberty   DraftKings  1.04            96.153846        0.961538
          WNBA         New York Liberty vs Atlanta Dream    h2h         Atlanta Dream  MyBookie.ag  7.75            12.903226        0.129032
          WNBA         New York Liberty vs Atlanta Dream    h2h      New York Liberty  MyBookie.ag  1.07            93.457944        0.934579
          WNBA         New York Liberty vs Atlanta Dream    h2h         Atlanta Dream      FanDuel  9.70            10.309278        0.103093
          WNBA         New York Liberty vs Atlanta Dream    h2h      New York Liberty      FanDuel  1.05            95.238095        0.952381
      